In [1]:
try:
    import vectorbt
except ModuleNotFoundError as e:
    !pip install vectorbt    

try:
    import yfinance
except ModuleNotFoundError as e:
    !pip install yfinance   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

try:
    import talib
except ModuleNotFoundError as e:
    !pip install talib_binary         


from IPython.display import clear_output
clear_output()

!exit

In [5]:

import vectorbt as vbt
import talib
import pandas as pd


In [285]:
#@title 回測設定
symbol_name = "SOL/USDT" #@param {type:"string"}

#timeframe = ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
timeframe = ['1h','4h','1d']

date_start = "2021-01-01" #@param {type:"date"}
date_end = "2022-01-01" #@param {type:"date"}

ema_s_start = 12 #@param {type:"integer"}
ema_s_end = 14 #@param {type:"integer"}

ema_m_start = 50 #@param {type:"integer"}
ema_m_end = 53 #@param {type:"integer"}

sma_deduct_start = 20 #@param {type:"integer"}
sma_deduct_end = 23 #@param {type:"integer"}


init_cash = 100 #@param {type:"integer"}



In [ ]:
result = pd.DataFrame()
for i_timeframe in timeframe:
    data = vbt.CCXTData.download(
        symbols=symbol_name,
        start=date_start,
        end=date_end,
        timeframe=i_timeframe)

    ohlc_df = data.get("Close")
    
    for i_ema_s in range(ema_s_start , ema_s_end):
        #print(f"emasss:  {i_ema_s}")

        for i_ema_m in range(ema_m_start , ema_m_end):
            #print(f"emammm:  {i_ema_m}")
            ema_sline = talib.EMA(ohlc_df,timeperiod=i_ema_s)
            ema_mline = talib.EMA(ohlc_df,timeperiod=i_ema_m)
                
            ema_entry = [False]
            ema_exit = [False]
                
            for i in range(1,ema_sline.shape[0]):
                if ema_sline[i] > ema_mline[i] and ema_sline[i-1] < ema_mline[i-1]:
                    ema_entry.append(True)
                else:
                    ema_entry.append(False)

                if ema_sline[i] < ema_mline[i] and ema_sline[i-1] > ema_mline[i-1]:
                    ema_exit.append(True)
                else:
                    ema_exit.append(False)

            ema_pf = vbt.Portfolio.from_signals(
                    close=ohlc_df, 
                    entries=ema_entry,
                    exits=ema_exit, 
                    init_cash=init_cash)
            ema_portfolio = ema_pf.stats()
            ema_sr = pd.Series([i_timeframe,i_ema_s,i_ema_m,0],index=["timeframe","ema_s","ems_m","sma_deduct"]) 
            ema_portfolio = pd.concat([ema_sr,ema_portfolio])
            result = pd.concat([result,ema_portfolio.to_frame().T],axis=0,ignore_index=True)   
            
                        
            for i_sma_d in range(sma_deduct_start,sma_deduct_end):
                sma_d_exit = [False]
                for i in range(1,ohlc_df.shape[0]):
                    if ohlc_df[i] < ohlc_df[i-i_sma_d]:
                        sma_d_exit.append(True)
                    else:
                        sma_d_exit.append(False)    
                
                sma_pf = vbt.Portfolio.from_signals(
                    close=ohlc_df, 
                    entries=ema_entry,
                    exits=sma_d_exit, 
                    init_cash=init_cash)       

                sma_portfolio = sma_pf.stats() 
                sma_sr = pd.Series([i_timeframe,i_ema_s,i_ema_m,i_sma_d],index=["timeframe","ema_s","ems_m","sma_deduct"])
                sma_portfolio = pd.concat([sma_sr,sma_portfolio])
                result = pd.concat([result,sma_portfolio.to_frame().T],axis=0,ignore_index=True)   
           

clear_output() 
print(f"backtimes: {result.shape[0]}")   

In [ ]:
#依獲利排序，取前100, 存到csv檔
result.sort_values("End Value", ascending=False, inplace=True)
result.head(100).to_csv(f"{symbol_name.replace('/','')}_fulltable.csv")
result.head(100)

In [ ]:
#依獲利排序，取前100, 存到csv檔
result.sort_values("Win Rate [%]", ascending=False, inplace=True)
result.head(100).to_csv(f"{symbol_name.replace('/','')}_fulltable.csv")
result.head(100)